In [4]:
import duckdb
con = duckdb.connect('mimiciv.duckdb')


In [5]:
query = """
-- 创建 dka_cohort 视图，供后续步骤使用
CREATE OR REPLACE VIEW dka_cohort AS
WITH dka_hadm_ids AS (
    SELECT DISTINCT hadm_id
    FROM diagnoses_icd
    WHERE REPLACE(icd_code, '.', '') IN (
        '25010', '25011', '25012',  -- ICD-9
        'E1010', 'E1110', 'E1310'   -- ICD-10
    )
),
dka_icu AS (
    SELECT 
        i.subject_id,
        i.hadm_id,
        i.stay_id,
        i.intime,
        i.outtime,
        p.anchor_age AS age,
        p.gender
    FROM icustays i
    INNER JOIN dka_hadm_ids d ON i.hadm_id = d.hadm_id
    INNER JOIN admissions a ON i.hadm_id = a.hadm_id
    INNER JOIN patients p ON i.subject_id = p.subject_id
    WHERE p.anchor_age >= 18
)
SELECT * FROM dka_icu
"""
con.execute(query)


In [6]:
# 查看前几行
con.execute("SELECT * FROM dka_cohort LIMIT 5").df()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,subject_id,hadm_id,stay_id,intime,outtime,age,gender
0,10924444,20065893,31555593,2167-04-27 04:21:00,2167-04-28 22:58:25,23,M
1,11036853,25681774,36673761,2145-05-05 01:19:47,2145-05-08 18:06:17,53,F
2,11213546,26189768,34015336,2183-05-16 07:08:12,2183-05-27 17:10:02,62,F
3,11306818,23614740,31826009,2167-12-10 21:06:00,2167-12-12 14:37:13,72,F
4,10015860,25085565,32496174,2186-09-15 17:15:00,2186-09-16 11:17:40,53,M


In [7]:
# 查看总共筛选出多少名 DKA + ICU 成年患者
con.execute("SELECT COUNT(*) FROM dka_cohort").fetchone()[0]


1400

In [8]:
con.execute("SELECT MIN(age), MAX(age) FROM dka_cohort").df()


,min(age),max(age)
0,18,91


In [9]:
con.execute("SELECT gender, COUNT(*) FROM dka_cohort GROUP BY gender").df()


,gender,count_star()
0,M,713
1,F,687
